# Importando as bibliotecas

In [ ]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from db import connection_db as conndb
from db import filters
from tqdm.auto import tqdm

tqdm.pandas()

Conectando ao banco de dados do MongoDB

In [ ]:
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
col_name = 'dadosSemFiltros'

mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_name)
mongo_connection.connect()
collection = mongo_connection.collection

Aplicando pipeline para preparação dos dados

In [ ]:
collection_filters = filters.CollectionFilters(collection)

# Aplicando pipeline 1
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')

# Aplicando pipeline 2 - pega os posts que tenha a data de publicação de no máximo 7 anos e no mínimo 2 anos
collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')

# Aplicando pipeline 3 - desenrola os posts em documentos individuais e cria novas colunas
collection_filters.apply_pipeline3('postsComBDIAndInfos')

# Aplicando pipeline 4 - aplica o filtro para selecionar apenas os posts que tem uma data de publicação válida
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')

collection = collection_filters.collection

In [ ]:
documentos = collection.find()
df = pd.DataFrame(list(documentos))

In [ ]:
df.head()

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

# Preencher os NaNs com 0
df[colunas_para_converter] = df[colunas_para_converter].fillna(0)

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

# Filtragem dos dados

In [ ]:
df.drop(columns=['_id', 'diaDaSemana', 'hora', 'minutos', 'diaDoMes', 'mes', 'ano'], inplace=True)

In [ ]:
df['data'] = df['postCreatedTime'].dt.date
df['data'] = pd.to_datetime(df['data'])

# EDA com filtro de 11 meses

In [ ]:
# Filtrar posts do ano de 2016
df_2017 = df[df['data'].dt.year == 2017]
df_2017.describe()

In [ ]:
# Remover posts de dezembro de 2017
df_2017 = df_2017[~((df_2017['data'].dt.month == 12) & (df_2017['data'].dt.year == 2017))]

In [ ]:
# Contar quantos usuários postaram em 2017

niveis = [1, 2, 3]

for nivel in niveis:
    # Filtrar o DataFrame para o nível de suicida atual
    df_nivel = df_2017[df_2017['suicida'] == nivel]

    # Agrupar por sexo e contar quantos usuários únicos existem por sexo no nível atual
    usuarios_por_genero = df_nivel.groupby('sexo')['id_usuario'].nunique()

    # Plotar gráfico de barras para o nível atual
    ax = usuarios_por_genero.plot(kind='bar', figsize=(8, 6), color=['skyblue', 'lightcoral'])
    plt.title(f'Quantidade de Usuários por Sexo e Nível de Ideação Suicida {nivel}')
    plt.xlabel('Gênero')
    plt.ylabel('Número de Usuários')
    plt.xticks(rotation=0)

    # Adicionar os valores no topo de cada barra
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    # Exibir o gráfico
    plt.show()

In [ ]:
df_2017 = df_2017[df_2017['suicida'] == 3]

In [ ]:
# Contar quantos posts cada usuário fez por dia
posts_por_dia = df_2017.groupby(['id_usuario', 'data']).size().reset_index(name='num_posts')

posts_por_dia.to_csv('posts_por_dia.csv', index=False)

# Exibir os dados
posts_por_dia.head()

In [ ]:
# Criando a coluna de 'mes'

posts_por_dia['mes'] = posts_por_dia['data'].dt.to_period('M')

# Agrupar por 'id_usuario' e 'mes' para contar os posts por mês para cada usuário
posts_por_mes_usuario = posts_por_dia.groupby(['id_usuario', 'mes'])['num_posts'].sum().reset_index()

# Plotar gráficos de linha separados para cada usuário
for usuario in posts_por_mes_usuario['id_usuario'].unique():
    df_usuario = posts_por_mes_usuario[posts_por_mes_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['mes'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Mês')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

posts_por_dia['semana'] = posts_por_dia['data'].dt.to_period('W')

# Agrupar por 'id_usuario' e 'semana' para contar os posts por semana para cada usuário
posts_por_semana_usuario = posts_por_dia.groupby(['id_usuario', 'semana'])['num_posts'].sum().reset_index()

for usuario in posts_por_semana_usuario['id_usuario'].unique():
    df_usuario = posts_por_semana_usuario[posts_por_semana_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['semana'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por Semana - Usuário {usuario}')
    plt.xlabel('Semana')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

# EDA com filtro de 6 meses

In [ ]:
# Passo 1: Filtrar dados após maio de 2017
df_2017 = df_2017[df_2017['data'].dt.month >= 6]

# Passo 2: Criar uma coluna 'mes' para facilitar o agrupamento
df_2017['mes'] = df_2017['data'].dt.to_period('M')

# Passo 3: Obter a lista de meses únicos no dataset
meses_unicos = df_2017['mes'].unique()
num_meses = len(meses_unicos)

# Passo 4: Contar o número de meses em que cada usuário tem pelo menos um post
posts_por_usuario_mes = df_2017.groupby(['id_usuario', 'mes']).size().reset_index(name='num_posts')

# Passo 5: Contar o número total de meses ativos para cada usuário
meses_ativos_por_usuario = posts_por_usuario_mes.groupby('id_usuario')['mes'].nunique().reset_index(name='meses_ativos')

# Passo 6: Filtrar usuários que têm posts em todos os meses (pelo menos um post por mês)
usuarios_com_posts_todos_meses = meses_ativos_por_usuario[meses_ativos_por_usuario['meses_ativos'] == num_meses]

# Exibir os usuários que têm posts em todos os meses
print(usuarios_com_posts_todos_meses)

# Filtrar o DataFrame original para conter apenas esses usuários
df_2017 = df_2017[df_2017['id_usuario'].isin(usuarios_com_posts_todos_meses['id_usuario'])]

# Exibir o DataFrame filtrado
df_2017.head()

In [ ]:
df_2017.to_csv('df_2017_filtro_6m.csv', index=False)

In [ ]:
# Contar quantos posts por dia cada usuário fez nesses 6 meses
posts_por_dia = df_2017.groupby(['id_usuario', 'data']).size().reset_index(name='num_posts')

posts_por_dia.to_csv('posts_por_dia_filtro_6m.csv', index=False)

# Exibir os dados
posts_por_dia.head()

In [ ]:
# Criar a coluna de 'mes'
posts_por_dia['mes'] = posts_por_dia['data'].dt.to_period('M')

# Agrupar por 'id_usuario' e 'mes' para contar os posts por mês para cada usuário
posts_por_mes_usuario = posts_por_dia.groupby(['id_usuario', 'mes'])['num_posts'].sum().reset_index()

# Plotar gráficos de linha separados para cada usuário
for usuario in posts_por_mes_usuario['id_usuario'].unique():
    df_usuario = posts_por_mes_usuario[posts_por_mes_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['mes'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Mês')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

In [ ]:
posts_por_dia['semana'] = posts_por_dia['data'].dt.to_period('W')

# Agrupar por 'id_usuario' e 'semana' para contar os posts por semana para cada usuário
posts_por_semana_usuario = posts_por_dia.groupby(['id_usuario', 'semana'])['num_posts'].sum().reset_index()

for usuario in posts_por_semana_usuario['id_usuario'].unique():
    df_usuario = posts_por_semana_usuario[posts_por_semana_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    plt.figure(figsize=(10, 6))
    plt.plot(df_usuario['semana'].astype(str), df_usuario['num_posts'], marker='o')
    plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    plt.xlabel('Semana')
    plt.ylabel('Número de posts')
    plt.xticks(rotation=90)
    plt.grid(True)

    # Exibir o gráfico
    plt.show()

# Decomposição sazonal individualmente

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# 
# periodo = 30
# 
# # Passo 2: Iterar sobre cada usuário
# for usuario in df_2017['id_usuario'].unique():
# 
#     # Filtrar o DataFrame para o usuário atual
#     df_usuario = df_2017[df_2017['id_usuario'] == usuario]
# 
#     # Passo 3: Agrupar por data para contar o número de posts por dia
#     # Preencher datas faltantes com 0
#     posts_por_dia = df_usuario.groupby('data').size().reindex(pd.date_range(df_usuario['data'].min(),
#                                                                             df_usuario['data'].max(), freq='D'),
#                                                               fill_value=0)
# 
#     # Verificar se há observações suficientes para decomposição (>= 2 * periodo)
#     if len(posts_por_dia) >= 2 * periodo:
#         # Aplicar a decomposição sazonal
#         try:
#             resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)
# 
#             # Passo 5: Plotar os componentes da decomposição
#             fig = plt.figure(figsize=(8, 6))
#             resultado.plot()
#             plt.suptitle(f'Decomposição da Série Temporal - Usuário {usuario}', y=1.02)
#             plt.show()
# 
#         except ValueError as e:
#             print(f"Não foi possível decompor a série para o usuário {usuario}: {e}")
#     else:
#         print(
#             f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import pandas as pd

periodo = 30

# Lista para armazenar DataFrames de tendências de cada usuário
lista_tendencias = []

# Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Preparar dados para a regressão linear
            X = np.arange(len(tendencia)).reshape(-1, 1)
            y = tendencia.values  # Valores da tendência

            # Aplicar a Regressão Linear
            model = LinearRegression()
            model.fit(X, y)

            # Inclinação (slope) da tendência
            slope = model.coef_[0]

            X_sm = sm.add_constant(X)
            model_sm = sm.OLS(y, X_sm).fit()
            # print(model_sm.summary())

            r_squared = model.score(X, y)
            print(f'R² da regressão para o usuário {usuario}: {r_squared:.4f}')

            # Exibir o resultado
            print(f'Inclinação (slope) da tendência para o usuário {usuario}: {slope}')

            result = adfuller(y)
            print('Estatística ADF:', result[0])
            print('P-valor:', result[1])

            # Armazenar a tendência em um DataFrame
            df_tendencia = tendencia.reset_index()
            df_tendencia.columns = ['Data', 'Tendência']
            df_tendencia['id_usuario'] = usuario  # Adicionar coluna com o ID do usuário

            # Adicionar o DataFrame à lista
            lista_tendencias.append(df_tendencia)

            # Plotar os dados e a linha de tendência ajustada
            plt.figure(figsize=(15, 12))
            plt.plot(tendencia.index, tendencia.values, label='Tendência Original')
            plt.plot(tendencia.index, model.predict(X), label='Tendência Ajustada (Regressão Linear)', linestyle='--')
            plt.title(f'Tendência para o Usuário {usuario}')
            plt.xlabel('Data')
            plt.ylabel('Tendência')
            plt.legend()
            plt.grid(True)
            plt.show()

            # Plotar a tendência do usuário atual
            plt.figure(figsize=(15, 10))
            plt.plot(tendencia.index, tendencia.values)
            plt.title(f'Tendência de Postagens para o Usuário {usuario}')
            plt.xlabel('Data')
            plt.ylabel('Número de Postagens')
            plt.show()

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, concatenar todas as tendências em um único DataFrame
df_todas_tendencias = pd.concat(lista_tendencias, ignore_index=True)

# Exibir as primeiras linhas do DataFrame resultante
print(df_todas_tendencias.head())

# Opcional: Salvar o DataFrame em um arquivo CSV
df_todas_tendencias.to_csv('tendencias_todos_usuarios.csv', index=False)

# Plotar as tendências de todos os usuários
# import seaborn as sns
# 
# plt.figure(figsize=(20, 15))
# sns.lineplot(data=df_todas_tendencias, x='Data', y='Tendência', hue='id_usuario')
# plt.title('Comparação das Tendências entre Usuários')
# plt.xlabel('Data')
# plt.ylabel('Número de Postagens (Tendência)')
# plt.legend(title='ID do Usuário')
# plt.show()

In [ ]:
import pandas as pd
from pymannkendall import original_test
from statsmodels.tsa.seasonal import seasonal_decompose

periodo = 30

# Lista para armazenar os resultados
resultados_mk = []

# Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Aplicar o teste de Mann-Kendall
            result = original_test(tendencia)
            print(result, '\n')

            # Armazenar os resultados em um dicionário
            resultados_mk.append({
                'id_usuario': usuario,
                'trend': result.trend,
                'h': result.h,
                'p-valor': result.p,
                'z': result.z,
                'Tau': result.Tau,
                's': result.s,
                'var_s': result.var_s,
                'slope': result.slope,
                'intercept': result.intercept
            })

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Após o loop, converter a lista de resultados em um DataFrame
df_resultados_mk = pd.DataFrame(resultados_mk)

# Exibir as primeiras linhas do DataFrame
df_resultados_mk.head()

# Opcional: Salvar o DataFrame em um arquivo CSV
df_resultados_mk.to_csv('resultados_mann_kendall.csv', index=False)

# Contar o número de usuários em cada tipo de tendência
contagem_tendencias = df_resultados_mk['trend'].value_counts()
print(contagem_tendencias)

# Calcular percentuais
total_usuarios = df_resultados_mk.shape[0]
percentuais = (contagem_tendencias / total_usuarios) * 100
print(percentuais)

import seaborn as sns

# Gráfico de barras das tendências
plt.figure(figsize=(8, 6))
sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
plt.title('Distribuição de Tendências entre Usuários')
plt.xlabel('Tipo de Tendência')
plt.ylabel('Número de Usuários')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
plt.title('Distribuição dos Slopes entre Usuários')
plt.xlabel('Slope')
plt.ylabel('Frequência')
plt.show()

# Filtrar usuários com tendência crescente significativa
usuarios_crescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'increasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência crescente significativa: {len(usuarios_crescentes)}")

# Filtrar usuários com tendência decrescente significativa
usuarios_decrescentes = df_resultados_mk[(df_resultados_mk['trend'] == 'decreasing') & (df_resultados_mk['h'] == True)]
print(f"Usuários com tendência decrescente significativa: {len(usuarios_decrescentes)}")

# Filtrar usuários sem tendência significativa
usuarios_sem_tendencia = df_resultados_mk[(df_resultados_mk['trend'] == 'no trend') & (df_resultados_mk['h'] == False)]
print(f"Usuários sem tendência significativa: {len(usuarios_sem_tendencia)}")

In [ ]:
usuarios_decrescentes.head()

In [ ]:
from scipy import fftpack
from statsmodels.graphics.tsaplots import plot_acf

periodo = 30

# Lista para armazenar os resultados
resultados_mk = []

# Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Aplicando a Transformada Rápida de Fourier
            tendencia_fft = fftpack.fft(tendencia.values)

            # Frequências correspondentes
            freqs = fftpack.fftfreq(len(tendencia.values))

            # Plotando o espectro de frequências
            # plt.figure(figsize=(10, 6))
            # plt.plot(freqs, np.abs(tendencia_fft))
            # plt.title('Análise Espectral da Tendência')
            # plt.xlabel('Frequência')
            # plt.ylabel('Amplitude')
            # plt.show()
            # 
            # plot_acf(tendencia.values, lags=30)
            # plt.title('Função de Autocorrelação da Tendência')
            # plt.show()

            # Aplicar o teste de Mann-Kendall
            result = original_test(tendencia)
            print(result, '\n')

            # Armazenar os resultados em um dicionário
            resultados_mk.append({
                'id_usuario': usuario,
                'trend': result.trend,
                'h': result.h,
                'p-valor': result.p,
                'z': result.z,
                'Tau': result.Tau,
                's': result.s,
                'var_s': result.var_s,
                'slope': result.slope,
                'intercept': result.intercept
            })

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

In [ ]:
df_2017.head()

In [ ]:
df_resultados_mk.head()

In [ ]:
df_trend_decreasing = df_resultados_mk[df_resultados_mk['trend'] == 'decreasing'].copy()

usuarios_decreasing = df_trend_decreasing['id_usuario']

# Exibir os resultados
print(usuarios_decreasing.head())

In [ ]:
# filtrar no dataframe df_2017 apenas os usuarios presentes no dataframe usuarios_decreasing
df_2017_decreasing = df_2017[df_2017['id_usuario'].isin(usuarios_decreasing)].copy()

In [ ]:
df_2017_decreasing.head()

In [ ]:
from utils.estracao_interacao import ExtracaoInteracao

extracao = ExtracaoInteracao(df_2017_decreasing)
df_with_interactions = extracao.extract_interactions()

# Passo 6: Modificar outras colunas usando .loc para evitar avisos
df_with_interactions.loc[:, 'sexo'] = df_with_interactions['sexo'].map({'F': 0, 'M': 1})
df_with_interactions.loc[:, 'diaDaSemanaNumerico'] = df_with_interactions['postCreatedTime'].dt.weekday

In [ ]:
df_with_interactions.head()

In [ ]:
from utils.text_treatment import TextTreatment

tratamento_texto = TextTreatment()
df_with_interactions['postMessageLimpo'] = df_with_interactions['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')
from utils.busca_palavras import BuscaPalavras

busca_palavras = BuscaPalavras()

# Faz a busca exata
resultado = busca_palavras.string_matching(df['postMessageLimpo'], 'dados/datasets/termos_depressivos_pt_br.txt')

# Adicionar a coluna quantPalavrasDepressivas ao DataFrame, contando quantas palavras depressivas foram encontradas
df_with_interactions['quantPalavrasDepressivas'] = resultado.apply(lambda x: len(x.split(', ')) if x else 0)

In [ ]:
df_with_interactions.head()